In [25]:
def generate_text(model, length, vocab_size, ix_to_word):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_word = [ix_to_word[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_word[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_word.append(ix_to_word[ix[-1]])
    return ('').join(y_word)

# method for preparing the training data
def load_data(data, seq_length):
    vocab = []
    word = []
    for i in range(len(data)):
        for j in range(len(data[i])):
            word.append(data[i][j])
            if data[i][j] not in vocab:
                vocab.append(data[i][j])
    #chars = list(set(data))
    VOCAB_SIZE = len(vocab)

    print('Data length: {} words'.format(len(word)))
    print('Vocabulary size: {} words'.format(VOCAB_SIZE))

    ix_to_word = {ix:word for ix, word in enumerate(vocab)}
    word_to_ix = {word:ix for ix, word in enumerate(vocab)}

    X = np.zeros((int(len(word)/seq_length), seq_length, VOCAB_SIZE))
    y = np.zeros((int(len(word)/seq_length), seq_length, VOCAB_SIZE))
    for i in range(0, int(len(data)/seq_length)):
        X_sequence = word[i*seq_length:(i+1)*seq_length]
        X_sequence_ix = [word_to_ix[value] for value in X_sequence]
        input_sequence = np.zeros((seq_length, VOCAB_SIZE))
        for j in range(seq_length):
            input_sequence[j][X_sequence_ix[j]] = 1.
            X[i] = input_sequence

        y_sequence = word[i*seq_length+1:(i+1)*seq_length+1]
        y_sequence_ix = [word_to_ix[value] for value in y_sequence]
        target_sequence = np.zeros((seq_length, VOCAB_SIZE))
        for j in range(seq_length):
            target_sequence[j][y_sequence_ix[j]] = 1.
            y[i] = target_sequence
    return X, y, VOCAB_SIZE, ix_to_word

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
import argparse
import pandas as pd
import collections
import nltk
#from RNN_utils import *


articles1 = pd.read_csv('articles1.csv')
articles = articles1.iloc[:100,:]

def cleaner(x):
    x = nltk.word_tokenize(str(x))
    return x

articles.content = articles.content.map(lambda x:cleaner(x))
articles.title = articles.title.map(lambda x:cleaner(x))

maxlen = 0
word_freqs = collections.Counter()
unique = []
num_recs = 0
for line in range(len(articles.content)):
    words = articles.content[line]
    if len(words)>maxlen:
        maxlen = len(words)
    for word in words:
        word_freqs[word] += 1
        if word not in unique:
            unique.append(word)
    num_recs += 1


# Creating training data
X, y, VOCAB_SIZE, ix_to_word = load_data(articles.content, maxlen)

# Creating and compiling the Network
model = Sequential()
model.add(LSTM(1200, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(2):
  model.add(LSTM(600, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

# Generate some sample before training to know how bad it is!
generate_text(model, 1, VOCAB_SIZE, ix_to_word)

if not WEIGHTS == '':
  model.load_weights(WEIGHTS)
  nb_epoch = int(WEIGHTS[WEIGHTS.rfind('_') + 1:WEIGHTS.find('.')])
else:
  nb_epoch = 0

# Training if there is no trained weights specified
if args['mode'] == 'train' or WEIGHTS == '':
  while True:
    print('\n\nEpoch: {}\n'.format(nb_epoch))
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, nb_epoch=1)
    nb_epoch += 1
    generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
    if nb_epoch % 10 == 0:
      model.save_weights('checkpoint_layer_{}_hidden_{}_epoch_{}.hdf5'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch))

# Else, loading the trained weights and performing generation only
elif WEIGHTS == '':
  # Loading the trained weights
  model.load_weights(WEIGHTS)
  generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_word)
  print('\n\n')
else:
  print('\n\nNothing to do!')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Data length: 146709 words
Vocabulary size: 15144 words
Knox

NameError: name 'WEIGHTS' is not defined

In [17]:
len(articles.content[0])

5607